In [1]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision

In [2]:
EPOCH = 1
BATCH_SIZE = 50
LR = 0.001
DOWNLOAD_MINST = False
train_data = torchvision.datasets.MNIST(
    root = './mnist/',
    train = True,
    transform = torchvision.transforms.ToTensor(),
    download = DOWNLOAD_MINST
)

In [3]:
train_loader = Data.DataLoader(dataset=train_data,batch_size=BATCH_SIZE,shuffle=True)
test_data = torchvision.datasets.MNIST(root='./mnist/',train=False)

In [15]:
test_x=torch.unsqueeze(test_data.test_data,dim=1).type(torch.FloatTensor)[:2000]
test_y=test_data.test_labels[:2000]
test_loader = Data.DataLoader(dataset=test_data,batch_size=2000,shuffle=True)
test_x = test_x.cuda()
test_y = test_y.cuda()

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1,16,5,1,2),#[1,28,28] => [16,28,28]
            nn.ReLU(),
            nn.MaxPool2d(2) #[16,14,14]
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16,32,5,1,2),#[16,14,14] => [32,14,14]
            nn.ReLU(),
            nn.MaxPool2d(2) #[32,7,7]
        )
        self.out = nn.Linear(32*7*7,10)
    
    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0),-1)
        output = self.out(x)
        return output
cnn =CNN()
cnn.cuda()
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)


In [50]:
opt = torch.optim.Adam(cnn.parameters(),lr=LR)
loss_func = nn.CrossEntropyLoss()
for epoch in range(EPOCH):
    for step,(x,y) in enumerate(train_loader):
        b_x = x.cuda()
        b_y = y.cuda()
        output = cnn(b_x)
        loss = loss_func(output,b_y)
        opt.zero_grad()
        loss.backward()
        opt.step()
        if step%100 == 0:
            test_output = cnn(test_x)
            print(torch.max((nn.functional.softmax(test_output,dim=1)),1)[1])
            pred_y = torch.max(nn.functional.softmax(test_output,dim=1),1)[1]#.cuda().data.squeeze()
            accuracy = sum(pred_y.cpu().numpy()==test_y.cpu().numpy())/test_y.size(0)
            print('Epoch:',epoch,'step:',step,'| train loss:',loss.data.cpu().numpy(),
                 '| test accuracy:',accuracy
                 )

tensor([7, 2, 1,  ..., 3, 9, 5], device='cuda:0')
Epoch: 0 step: 0 | train loss: 0.00020557403 | test accuracy: 0.987
tensor([7, 2, 1,  ..., 3, 9, 5], device='cuda:0')
Epoch: 0 step: 100 | train loss: 0.003673327 | test accuracy: 0.988
tensor([7, 2, 1,  ..., 3, 9, 5], device='cuda:0')
Epoch: 0 step: 200 | train loss: 0.00068756344 | test accuracy: 0.9875
tensor([7, 2, 1,  ..., 3, 9, 5], device='cuda:0')
Epoch: 0 step: 300 | train loss: 1.749754e-05 | test accuracy: 0.9835
tensor([7, 2, 1,  ..., 3, 9, 5], device='cuda:0')
Epoch: 0 step: 400 | train loss: 0.0021593166 | test accuracy: 0.986
tensor([7, 2, 1,  ..., 3, 9, 5], device='cuda:0')
Epoch: 0 step: 500 | train loss: 0.00012758971 | test accuracy: 0.987
tensor([7, 2, 1,  ..., 3, 9, 5], device='cuda:0')
Epoch: 0 step: 600 | train loss: 0.00066326855 | test accuracy: 0.9875
tensor([7, 2, 1,  ..., 3, 9, 5], device='cuda:0')
Epoch: 0 step: 700 | train loss: 0.0008010483 | test accuracy: 0.982
tensor([7, 2, 1,  ..., 3, 9, 5], device='cud

In [36]:
import numpy as np

In [48]:
np.max(([[1,2,3,0],[4,1,2,1]]),1)[1]

4

In [54]:
test_output = cnn(test_x[1990:2000])
pred_y = torch.max(test_output, 1)[1]#.data.squeeze()
print(pred_y.cpu().numpy(), 'prediction number')
print(test_y[1990:2000].cpu().numpy(), 'real number')

[6 2 9 1 1 0 6 3 9 5] prediction number
[6 2 9 1 1 0 6 3 9 5] real number
